In [1]:
from gensim.corpora import Dictionary
from gensim.models import Word2Vec

import pandas as pd

import pickle

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [4]:
word_freqs_vocabulary = pd.read_csv("../data/word_freqs_vocabulary.csv")

In [5]:
word_freqs_vocabulary

,word,count
0,λέγω,29114
1,ἔχω,22584
2,γίγνομαι,21737
3,πολύς,20652
4,πᾶς,19062
...,...,...
3475,στραγγουρία,50
3476,ῥύγχος,50
3477,διάκρισις,50
3478,ἀρχός,50


In [13]:
word_freq_dict = dict(zip(word_freqs_vocabulary["word"], word_freqs_vocabulary["count"]))

In [6]:
vocabulary = word_freqs_vocabulary["word"].tolist()

In [9]:
class NgramCorpus:
    def __iter__(self):
        with open("../data/large_data/corpus_ngrams.txt", "r") as f:
            lines = f.readlines()
            for line in lines:
                yield line.split()

In [11]:
corpus = NgramCorpus()
corpus_len = len([doc for doc in corpus])
corpus_len

3232119

In [20]:
%%time
model = Word2Vec(vector_size=150, window=3, negative=5, ns_exponent=1, sg=0, epochs=5, workers=8)
model.build_vocab_from_freq(word_freq=word_freq_dict)
model.train(corpus, total_examples=corpus_len, epochs=model.epochs)

CPU times: user 1min 1s, sys: 1.33 s, total: 1min 3s
Wall time: 37.9 s


(29277338, 45966425)

In [21]:
keyed_vectors = model.wv

In [22]:
keyed_vectors.save("../data/keyed_vectors.wv")